In [218]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [219]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_dict = {i: months[i-1] for i in range(0, 13)}

In [444]:
pd_data = pd.read_csv('pd_data_clean.csv')
pd_data = pd_data.dropna(subset=['Location'])
strs = []
for i in range(len(pd_data)):
    month = (pd_data.iloc[i]['MONTH'].astype(int))
    year = pd_data.iloc[i]['YEAR']
    strs.append(f'{year} {month}')
pd_data['DATE'] = strs
pd_data.head()

,Unnamed: 0,OFFENSE_CODE,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,THEME,SERIOUS,MILES_FROM_NU,DATE
0,0,3115,0,2023-01-27 22:44:00+00,2023,1,Friday,22,NaN,FAVRE ST,42.271661,-71.099535,"(42.271661031027065, -71.09953455161765)",REPORT,0,4.735747,2023 1
1,1,3301,0,2023-01-17 20:21:00+00,2023,1,Tuesday,20,NaN,HAROLD ST,42.312596,-71.092876,"(42.3125962960786, -71.09287583752645)",NOISY,0,1.890701,2023 1
2,2,3115,0,2023-01-24 00:00:00+00,2023,1,Tuesday,0,NaN,HANOVER ST,42.365699,-71.052892,"(42.3656993584582, -71.05289203191269)",REPORT,0,2.600130,2023 1
3,3,3114,0,2023-03-31 17:14:00+00,2023,3,Friday,17,NaN,BLUE HILL AVE,42.292788,-71.088520,"(42.29278799134922, -71.08851953962399)",REPORT,0,3.252513,2023 3
4,4,423,0,2023-01-26 09:00:00+00,2023,1,Thursday,9,NaN,HUTCHINGS ST,42.310269,-71.089311,"(42.310268910354935, -71.08931055058088)",ASSAULT,1,2.045410,2023 1


In [445]:
def get_point(loc):
    loc = str(loc)
    loc = loc.strip("()").split(',')
    lat, long = map(float, loc)
    point = Point(long, lat)
    return point

In [446]:
pd_data['Coordinates'] = pd_data['Location'].apply(get_point)
gdf = gpd.GeoDataFrame(pd_data, geometry='Coordinates')

In [447]:
boston_bg = gpd.read_file('2020_Census_Block_Groups_in_Boston.shp')

In [448]:
gdf = gdf.set_crs("EPSG:4326")
#gdf = gdf.to_crs('EPSG:2249')
boston_bg = boston_bg.set_crs('EPSG:2249', allow_override=True)
boston_bg = boston_bg.to_crs('EPSG:4326')

In [449]:
print(gdf.crs)  # Check CRS of the GeoDataFrame
print(boston_bg.crs)

EPSG:4326
EPSG:4326


In [450]:
joined_gdf = gpd.sjoin(gdf, boston_bg, how='right', predicate='within')

In [451]:
joined_df = joined_gdf.set_geometry('geometry', crs = 'EPSG:4326')

In [452]:
to_map = gpd.GeoDataFrame(joined_gdf.groupby(['DATE', 'geometry'], as_index = False)['SERIOUS'].sum())
to_map = to_map.set_geometry('geometry', crs = 'EPSG:4326')

In [453]:
to_map['DATE'] = pd.to_datetime(to_map['DATE'], format='%Y %m')
to_map['DATE'] = to_map['DATE'].dt.strftime('%Y-%m')

In [460]:
to_map.geometry

0        POLYGON ((-71.17702 42.2804, -71.17555 42.2803...
1        POLYGON ((-71.1726 42.27157, -71.17225 42.2719...
2        POLYGON ((-71.16631 42.26968, -71.16615 42.270...
3        POLYGON ((-71.16351 42.26259, -71.16345 42.262...
4        POLYGON ((-71.16739 42.2622, -71.16711 42.2650...
                               ...                        
12572    POLYGON ((-71.05469 42.28034, -71.05399 42.280...
12573    POLYGON ((-71.05147 42.28931, -71.05136 42.289...
12574    POLYGON ((-71.0475 42.29864, -71.04737 42.2987...
12575    POLYGON ((-71.06818 42.27134, -71.06809 42.271...
12576    POLYGON ((-71.07214 42.27531, -71.07208 42.275...
Name: geometry, Length: 12577, dtype: geometry

In [462]:
import plotly.express as px
import json


fig = px.choropleth_mapbox(to_map, geojson = to_map.geometry, color = "SERIOUS", 
                           mapbox_style="carto-positron", animation_frame = 'DATE')

In [ ]:
fig.update_layout(
    mapbox_center={"lat": 42.0, "lon": -71.0},  # Change to your region
    mapbox_zoom=5  # Adjust zoom level as necessary
)

In [ ]:
fig.show()